# Practical Exam: House sales

RealAgents is a real estate company that focuses on selling houses.

RealAgents sells a variety of types of house in one metropolitan area.

Some houses sell slowly and sometimes require lowering the price in order to find a buyer.

In order to stay competitive, RealAgents would like to optimize the listing prices of the houses it is trying to sell.

They want to do this by predicting the sale price of a house given its characteristics.

If they can predict the sale price in advance, they can decrease the time to sale.


## Data

The dataset contains records of previous houses sold in the area.

| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
| house_id    | Nominal. </br> Unique identifier for houses. </br>Missing values not possible. |
| city        | Nominal. </br>The city in which the house is located. One of 'Silvertown', 'Riverford', 'Teasdale' and 'Poppleton'. </br>Replace missing values with "Unknown". |
| sale_price  | Discrete. </br>The sale price of the house in whole dollars. Values can be any positive number greater than or equal to zero.</br>Remove missing entries. |
| sale_date   | Discrete. </br>The date of the last sale of the house. </br>Replace missing values with 2023-01-01. |
| months_listed  | Continuous. </br>The number of months the house was listed on the market prior to its last sale, rounded to one decimal place. </br>Replace missing values with mean number of months listed, to one decimal place. |
| bedrooms    | Discrete. </br>The number of bedrooms in the house. Any positive values greater than or equal to zero. </br>Replace missing values with the mean number of bedrooms, rounded to the nearest integer. |
| house_type   | Ordinal. </br>One of "Terraced" (two shared walls), "Semi-detached" (one shared wall), or "Detached" (no shared walls). </br>Replace missing values with the most common house type. |
| area      | Continuous. </br>The area of the house in square meters, rounded to one decimal place. </br>Replace missing values with the mean, to one decimal place. |


# Task 1

The team at RealAgents knows that the city that a property is located in makes a difference to the sale price. 

Unfortuntately they believe that this isn't always recorded in the data. 

Calculate the number of missing values of the `city`. 

 - You should use the data in the file "house_sales.csv". 

 - Your output should be an object `missing_city`, that contains the number of missing values in this column. 

In [3]:
# Libs
import pandas as pd
import numpy as np

# reading the data 
df = pd.read_csv('house_sales.csv')
print(df.head())
print('*'*30)

# checking the city column
print(df['city'].unique())
print('*'*30)

# replacing '--' with 'Unknown'
list_city = ['Silvertown', 'Riverford', 'Teasdale', 'Poppleton']
df.loc[~df['city'].isin(list_city),'city'] = 'Unknown'

missing_city = df.loc[df['city']=='Unknown'].shape[0]
print(f'Number of missing values in "city": {missing_city}')

   house_id        city  sale_price   sale_date  months_listed  bedrooms  \
0   1217792  Silvertown       55943  2021-09-12            5.4         2   
1   1900913  Silvertown      384677  2021-01-17            6.3         5   
2   1174927   Riverford      281707  2021-11-10            6.9         6   
3   1773666  Silvertown      373251  2020-04-13            6.1         6   
4   1258487  Silvertown      328885  2020-09-24            8.7         5   

      house_type         area  
0  Semi-detached  107.8 sq.m.  
1       Detached  498.8 sq.m.  
2       Detached  542.5 sq.m.  
3           Det.  528.4 sq.m.  
4       Detached  477.1 sq.m.  
******************************
['Silvertown' 'Riverford' 'Teasdale' 'Poppleton' '--']
******************************
Number of missing values in "city": 73


# Task 2 

Before you fit any models, you will need to make sure the data is clean. 

The table below shows what the data should look like. 

Create a cleaned version of the dataframe. 

 - You should start with the data in the file "house_sales.csv". 

 - Your output should be a dataframe named `clean_data`. 

 - All column names and values should match the table below.


| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
| house_id    | Nominal. </br> Unique identifier for houses. </br>Missing values not possible. |
| city        | Nominal. </br>The city in which the house is located. One of 'Silvertown', 'Riverford', 'Teasdale' and 'Poppleton' </br>Replace missing values with "Unknown". |
| sale_price  | Discrete. </br>The sale price of the house in whole dollars. Values can be any positive number greater than or equal to zero.</br>Remove missing entries. |
| sale_date   | Discrete. </br>The date of the last sale of the house. </br>Replace missing values with 2023-01-01. |
| months_listed  | Continuous. </br>The number of months the house was listed on the market prior to its last sale, rounded to one decimal place. </br>Replace missing values with mean number of months listed, to one decimal place. |
| bedrooms    | Discrete. </br>The number of bedrooms in the house. Any positive values greater than or equal to zero. </br>Replace missing values with the mean number of bedrooms, rounded to the nearest integer. |
| house_type   | Ordinal. </br>One of "Terraced", "Semi-detached", or "Detached". </br>Replace missing values with the most common house type. |
| area      | Continuous. </br>The area of the house in square meters, rounded to one decimal place. </br>Replace missing values with the mean, to one decimal place. |

In [5]:
## cleaning the data
clean_data = df.copy()
print('*** base condition of the dat ***\n', df.dtypes)
print('*'*30)
print('*** base condition of the data ***\n', df.isna().sum())
print('*'*30)
print('*** final condition of the data ***\n', df.shape)
print('*'*30)

# house_id
clean_data['house_id'] = df['house_id'].astype('category')
assert clean_data['house_id'].nunique() == clean_data.shape[0]

# city
clean_data['city'] = df['city'].astype('category')
list_city_updated = ['Silvertown', 'Riverford', 'Teasdale', 'Poppleton', 'Unknown']
assert df['city'].isin(list_city_updated).all()

# sale_price
clean_data = clean_data.loc[clean_data['sale_price']>=0]
assert (clean_data['sale_price'] >= 0).all()

# sale_date
clean_data['sale_date'] = pd.to_datetime(df['sale_date'])

# months_listed
clean_data['months_listed'] =  df['months_listed'].fillna(df['months_listed'].mean().round(1))

# bedrooms
assert (clean_data['bedrooms']>=0).all()

# house_type
print(f'unique values in "house_type" before cleaning:\n{df["house_type"].unique()}')
print('*'*30)
house_type_list = ["Terraced", "Semi-detached", "Detached"]
house_type_mapping = {'Terr.':"Terraced", 'Semi':"Semi-detached", 'Det.':"Detached"}
clean_data['house_type'] = df['house_type'].replace(house_type_mapping).astype('category')
assert clean_data['house_type'].isin(house_type_list).all()

# area 
clean_data['area'] = df['area'].str.strip('sq.m').astype(float).round(1)
assert (clean_data['area']>=0).all()

print('*** final condition of the data ***\n', clean_data.dtypes)
print('*'*30)
print('*** final condition of the data ***\n', clean_data.isna().sum())
print('*'*30)
print('*** final condition of the data ***\n', clean_data.shape)
print('*'*30)
clean_data.head()

*** base condition of the dat ***
 house_id           int64
city              object
sale_price         int64
sale_date         object
months_listed    float64
bedrooms           int64
house_type        object
area              object
dtype: object
******************************
*** base condition of the data ***
 house_id          0
city              0
sale_price        0
sale_date         0
months_listed    31
bedrooms          0
house_type        0
area              0
dtype: int64
******************************
*** final condition of the data ***
 (1500, 8)
******************************
unique values in "house_type" before cleaning:
['Semi-detached' 'Detached' 'Det.' 'Terraced' 'Semi' 'Terr.']
******************************
*** final condition of the data ***
 house_id               category
city                   category
sale_price                int64
sale_date        datetime64[ns]
months_listed           float64
bedrooms                  int64
house_type             category
a

,house_id,city,sale_price,sale_date,months_listed,bedrooms,house_type,area
0,1217792,Silvertown,55943,2021-09-12,5.4,2,Semi-detached,107.8
1,1900913,Silvertown,384677,2021-01-17,6.3,5,Detached,498.8
2,1174927,Riverford,281707,2021-11-10,6.9,6,Detached,542.5
3,1773666,Silvertown,373251,2020-04-13,6.1,6,Detached,528.4
4,1258487,Silvertown,328885,2020-09-24,8.7,5,Detached,477.1


# Task 3 

The team at RealAgents have told you that they have always believed that the number of bedrooms is the biggest driver of house price. 

Producing a table showing the difference in the average sale price by number of bedrooms along with the variance to investigate this question for the team.

 - You should start with the data in the file 'house_sales.csv'.

 - Your output should be a data frame named `price_by_rooms`. 

 - It should include the three columns `bedrooms`, `avg_price`, `var_price`. 

 - Your answers should be rounded to 1 decimal place.   

In [7]:
# task
price_by_rooms = df.groupby('bedrooms')['sale_price'].agg(
    avg_price=lambda x: x.mean(), 
    var_price=lambda x: x.var()
).reset_index().round(1)

print(price_by_rooms)

   bedrooms  avg_price     var_price
0         2    67076.4  5.652896e+08
1         3   154665.1  2.378289e+09
2         4   234704.6  1.725211e+09
3         5   301515.9  2.484328e+09
4         6   375741.3  3.924432e+09


# Task 4

Fit a baseline model to predict the sale price of a house.

 1. Fit your model using the data contained in “train.csv” </br></br>

 2. Use “validation.csv” to predict new values based on your model. You must return a dataframe named `base_result`, that includes `house_id` and `price`. The price column must be your predicted values.

In [9]:
# data for tarining and validation 
train_data = pd.read_csv('train.csv')
print(f' the shape of training data: {train_data.shape}\n')

val_data = pd.read_csv('validation.csv')
print(f' the shape of validation data: {val_data.shape}\n')

# cleaning the new datasets
def data_cleaner(df):
    # producing dummies for house_type and city
    city_dummies = pd.get_dummies(df['city'], drop_first=True)
    house_type_dummies = pd.get_dummies(df['house_type'], drop_first=True)
    # elimimnating the columns that are being binary coded, 
    # the date columns, unique identifiers.
    df = df.drop(['house_id', 'city', 'house_type', 'sale_date'], axis=1)
    # putting the data together with binary codes for 
    # house_tyep and city 
    df_final = pd.concat([df, city_dummies, house_type_dummies], axis = 1)
    return df_final

# producing the clean versions of train_data and val_data
train_data_clean = data_cleaner(train_data)
print(f' the final shape of training data: {train_data.shape}\n')

val_data_clean = data_cleaner(val_data)
print(f' the final shape of validation data: {val_data.shape}\n')
print('*'*30)
print(train_data_clean.dtypes)
print('*'*30)
print(val_data_clean.dtypes)

 the shape of training data: (1200, 8)

 the shape of validation data: (300, 7)

 the final shape of training data: (1200, 8)

 the final shape of validation data: (300, 7)

******************************
sale_price         int64
months_listed    float64
bedrooms           int64
area             float64
Riverford           bool
Silvertown          bool
Teasdale            bool
Semi-detached       bool
Terraced            bool
dtype: object
******************************
months_listed    float64
bedrooms           int64
area             float64
Riverford           bool
Silvertown          bool
Teasdale            bool
Semi-detached       bool
Terraced            bool
dtype: object


In [10]:
# Libs
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error as MSE

# X and y
X = train_data_clean.drop('sale_price', axis=1)
y = train_data_clean['sale_price']

# Linear Regression
lm = LinearRegression()
lm_mdl = lm.fit(X,y)
y_pred_ml = lm_mdl.predict(val_data_clean)

# validating the model using cross val(kf=10)
kf = KFold(n_splits=10, random_state=1, shuffle=True)
lm_cv_result = cross_val_score(
    lm, 
    X,
    y,
    cv=kf,
    scoring='neg_mean_squared_error'
)
print(f'mean Model RMSE is: {np.sqrt(-lm_cv_result).mean().round(1)}(±{np.sqrt(-lm_cv_result).std().round(1)})\n')

# task
base_result = pd.DataFrame(
    {
        'house_id':val_data['house_id'],
        'price':y_pred_ml
    }
)
print(f'the final DataFrame.head() is:\n\n{base_result.head()}')

mean Model RMSE is: 21664.1(±1593.2)

the final DataFrame.head() is:

   house_id          price
0   1331375  121527.827316
1   1630115  304386.625267
2   1645745  384760.100656
3   1336775  123976.268985
4   1888274  271186.199353


# Task 5

Fit a comparison model to predict the sale price of a house.

 1. Fit your model using the data contained in “train.csv” </br></br>

 2. Use “validation.csv” to predict new values based on your model. You must return a dataframe named `compare_result`, that includes `house_id` and `price`. The price column must be your predicted values.

In [12]:
# Libs
from sklearn.ensemble import RandomForestRegressor

# Random Forest model
ranf = RandomForestRegressor()
ranf_mdl = ranf.fit(X,y)
y_pred_ranf = ranf_mdl.predict(val_data_clean)

# validating the model using cross val(kf=10)
kf = KFold(n_splits=10, random_state=1, shuffle=True)
ranf_cv_result = cross_val_score(
    ranf, 
    X,
    y,
    cv=kf,
    scoring='neg_mean_squared_error'
)
print(f'mean Model RMSE is: {np.sqrt(-ranf_cv_result).mean().round(1)}(±{np.sqrt(-ranf_cv_result).std().round(1)})\n')

# task
compare_result = pd.DataFrame(
    {
        'house_id':val_data['house_id'],
        'price':y_pred_ranf
    }
)
print(f'the final DataFrame.head() is:\n\n{compare_result.head()}')

mean Model RMSE is: 14916.7(±1686.7)

the final DataFrame.head() is:

   house_id      price
0   1331375   82532.88
1   1630115  301598.99
2   1645745  407090.64
3   1336775  107222.20
4   1888274  267764.01
